# Mocking

vollkommen isoliertes Testen von einzelnen Objekten oder Funktionen    
Testen der Interaktion von Objekten: vorher Umgebung nachbilden  

Mock-Objekt: Programmteil als Platzhalter für echte Objekte  

Mock-Objekte implementieren die Schnittstellen, über die das zu testende Objekt auf seine Umgebung zugreift  
erwartete Methodenaufrufe müssen vollständig, mit den korrekten Parametern und in der erwarteten Reihenfolge durchgeführt werden  
liefert vorher zum Testfall passend festgelegte Werte zurück  
werden verwendet, um ein bestimmtes Verhalten nachzustellen  

Mock-Objekte ersetzen:
- andere Objekte die zu Testzwecken nur schwierig zu erstellen sind
- online Resourcen
- Frontend
- Backend
- ...

## Einsatz

Konkret sind Mock-Objekte sinnvoll, wenn das „echte“ Objekt

- von „unerwarteten Fehlern“ während der Tests nicht beschädigt werden soll,
- nicht deterministische Ergebnisse liefert (z. B. die aktuelle Uhrzeit oder die aktuelle Temperatur),
- Schwierigkeiten bei der Vorbereitung oder während der Ausführung bereitet (z. B. beim Testen von Benutzungsoberflächen),
- Verhalten zeigen soll, das nur schwer auszulösen ist (z. B. einen Netzwerkfehler),
- langsam oder sehr komplex ist (z. B. eine vollständige Datenbank, die vor jedem Test erst initialisiert werden müsste),
- noch nicht existiert (z. B. in größeren Software- oder Hardware-Entwicklungsprojekten),
- Informationen und Methoden ausschließlich zu Testzwecken (und nicht für seine eigentliche Aufgabe) zur Verfügung stellen müsste,
- nicht oder schwer rückgängig zu machende Prozesse anstößt (z. B. Dateien einer Netzwerkressource löscht).

## Python Mocking-Frameworks

- Mock
- pymox
- dingus

Seit Python 3 ist mock Teil der Standard Bibliothek von Python und wird wie folgt importiert:

In [7]:
from unittest import mock

Das Hauptobjekt, das vom Framework zur Verfügung gestellt wird ist "Mock".  
Dieses kann instanziiert werden ohne Argumente.

In [8]:
m = mock.Mock()

Dieses Objekt hat die Eigenschaft, Methoden und Attribute on the fly erstellen zu können, wenn sie benötigt werden.  
Der innere Aufbau sieht wie folgt aus:

In [9]:
dir(m)

['assert_any_call',
 'assert_called',
 'assert_called_once',
 'assert_called_once_with',
 'assert_called_with',
 'assert_has_calls',
 'assert_not_called',
 'attach_mock',
 'call_args',
 'call_args_list',
 'call_count',
 'called',
 'configure_mock',
 'method_calls',
 'mock_add_spec',
 'mock_calls',
 'reset_mock',
 'return_value',
 'side_effect']

Das Mock Objekt löst im gegensatz zu normalen Python-Objekten keinen AttributeError aus, wenn sie nach einem nicht existierenden Attribut gefragt werden. Es wird eine andere Instanz von Mock selbst zurückgegeben.

In [10]:
m.some_attribute

<Mock name='mock.some_attribute' id='140438117254648'>

Das Attribut, auf das der Zugriffversucht wurde, ist innerhalb des Objekts erstellt worden, und der Zugriff darauf gibt dasselbe Scheinobjekt wie zuvor zurück.

In [11]:
dir(m)

['assert_any_call',
 'assert_called',
 'assert_called_once',
 'assert_called_once_with',
 'assert_called_with',
 'assert_has_calls',
 'assert_not_called',
 'attach_mock',
 'call_args',
 'call_args_list',
 'call_count',
 'called',
 'configure_mock',
 'method_calls',
 'mock_add_spec',
 'mock_calls',
 'reset_mock',
 'return_value',
 'side_effect',
 'some_attribute']

In [12]:
m.some_attribute

<Mock name='mock.some_attribute' id='140438117254648'>

Mock-Objekte sind Callables.  
Sie können sowohl als Attribute als auch als Methoden fungieren. Beim Versuch, den Mock aufzurufen, gibt er einfach einen anderen Mock mit einem Namen zurück, der Klammern enthält, um seine aufrufbare Natur zu signalisieren

In [13]:
m.some_attribute()

<Mock name='mock.some_attribute()' id='140437882776432'>

Mock ist das perfekte Werkzeug, um andere Objekte oder Systeme zu imitieren.  
Um Mocks in Tests verwenden zu können, müssen sie sich jedoch genau wie das Original verhalten.  
Dafür müssen sinnvolle Werte zurückgegeben oder Operationen ausgeführt werden.

## Return
Das Einfachste, was ein Mock tun kann, ist, bei jedem Aufruf einen bestimmten Wert zurückzugeben.  
Dies wird über das return_value-Attribut eines Mock-Objekts konfiguriert.  

Jetzt wird kein Mock-Objekt mehr zurückgegeben, sondern nur der statische Wert, der im Attribut return_value gespeichert ist.  

Auch Callables wie Funktionen oder Objekte werden zurückgeben, aber nicht ausgeführt.

In [14]:
m.some_attribute.return_value = 42
m.some_attribute()

42

In [15]:
def print_answer():
    print("42")

m.some_attribute.return_value = print_answer
m.some_attribute()

<function __main__.print_answer()>

## Side effect
Um Werte zurückzugeben, die von einer Funktion stammen, muss das Attribut side_effect verwendet werden.  

Es akzeptiert drei verschiedene Varianten von Objekten: 
- Callables
- Iterables
- Exceptions

### Exception

In [16]:
m.some_attribute.side_effect = ValueError('A custom value error')
m.some_attribute()

ValueError: A custom value error

### Iterable
Wenn ein Iterable übergeben wird, wie z.B. einen Generator, eine einfache Liste, ein Tupel oder ähnliche Objekte, liefert der Mock die Werte dieses Iterables zurück.  
Er gibt bei nachfolgenden Aufrufen des Mocks jeden Wert zurück, der im Iterable enthalten ist.

In [17]:
m.some_attribute.side_effect = range(3)
m.some_attribute()

0

In [18]:
m.some_attribute()

1

In [19]:
m.some_attribute()

2

In [20]:
m.some_attribute()

StopIteration: 

### Callable
#### Funktion ohne Argumente

In [21]:
def print_answer():
    print("42")       
m.some_attribute.side_effect = print_answer
m.some_attribute()

42


#### Funktion mit Argumenten

In [22]:
def print_number(num):
    print("Number:", num)
    
m.some_attribute.side_effect = print_number
m.some_attribute.side_effect(5)

Number: 5


#### Klasse

In [23]:
class Number(object):
    def __init__(self, value):
        self._value = value
    def print_value(self):
        print("Value:", self._value)

m.some_attribute.side_effect = Number
n = m.some_attribute.side_effect(26)
n

In [24]:
n.print_value()

Value: 26


## Testen mit Mocks
Beim Umgang mit externen Objekten interessiert uns nur, OB eine Methode aufgerufen wurde und WELCHE PARAMETER der Aufrufer an das Objekt übergeben hat. Wir testen nicht, ob das entfernte Objekt das richtige Ergebnis zurückgibt, dies wird vom Mock gefälscht, der tatsächlich genau das Ergebnis liefert, das wir brauchen.

Der Zweck der von Mock-Objekten bereitgestellten Methoden besteht also darin, zu überprüfen, welche Methoden wir auf dem Mock selbst aufgerufen haben und welche Parameter wir im Aufruf verwendet haben.

In [25]:
import ipytest
import pytest
import sys
ipytest.autoconfig()

### Asserting calls
Das erste, was uns beim Umgang mit einem externen Objekt normalerweise interessiert, ist zu wissen, dass eine bestimmte Methode aufgerufen wurde. Python-Mocks stellen die Methode assert_called() bereit, um diese Bedingung zu überprüfen.  

Wir instanziieren die myobj.MyObj-Klasse, die ein externes Objekt erfordert. Die Klasse soll die Methode connect() des externen Objekts ohne Parameter aufrufen.

In [52]:
%%ipytest
def test_instantiation():
    external_obj = mock.Mock()
    MyObj(external_obj)
    external_obj.connect.assert_called()

.                                                                                            [100%]
1 passed in 0.00s


In [48]:
class MyObj():
    def __init__(self, repo):
        repo.connect()

SyntaxError: unexpected EOF while parsing (<ipython-input-48-248d851d4cb8>, line 2)

asserting calls überprüft außerdem die übergebenen Parameter  

Um dies zu zeigen, nehmen wir an, dass wir erwarten, dass die Methode  

MyObj.setup()  
setup(cache=True, max_connections=256)  
für das externe Objekt aufruft.

In [47]:
%%ipytest
def test_setup():
    external_obj = mock.Mock()
    obj = MyObj(external_obj)
    obj.setup()
    external_obj.setup.assert_called_with(cache=True, max_connections=256)

.                                                                                            [100%]
1 passed in 0.00s


In [44]:
class MyObj():
    def __init__(self, repo):
        self._repo = repo
        repo.connect()

    def setup(self):
        self._repo.setup(cache=True)

In [46]:
class MyObj():
    def __init__(self, repo):
        self._repo = repo
        repo.connect()

    def setup(self):
        self._repo.setup(cache=True, max_connections=256)

##### Das Mock Objekt bietet noch weitere Methoden:

https://docs.python.org/3/library/unittest.mock.html

In [51]:
m = mock.Mock()
dir(m)

['assert_any_call',
 'assert_called',
 'assert_called_once',
 'assert_called_once_with',
 'assert_called_with',
 'assert_has_calls',
 'assert_not_called',
 'attach_mock',
 'call_args',
 'call_args_list',
 'call_count',
 'called',
 'configure_mock',
 'method_calls',
 'mock_add_spec',
 'mock_calls',
 'reset_mock',
 'return_value',
 'side_effect']